In [3]:
using ForwardDiff
using Base.Test
using Plots

In [77]:
function solve_penalty(J,c;num_iter=100,μ=1)
    function print_stats()
        println("Final Value: $x")
        println("Final Cost: $(Q(x))")
        println("Final mu: ", μ)
        println("Final C: $(c(x))")
        println("k: $k/$(length(tau))")
        println("Iterations: $iter")
    end
    
    tau = logspace(1,-5,30)
    k = 1
    
    Q(x) = J(x) + μ/2*c(x)^2
    dQ(x) = ForwardDiff.gradient(Q,x)
    hQ(x) = ForwardDiff.hessian(Q,x)
    p(x) = -inv(hQ(x))*dQ(x)
    
    x_prev = ones(n)*Inf
    X = zeros(n,num_iter)
    cost = zeros(num_iter)
    P = zeros(n,num_iter)
    x = copy(x0)
        
    for iter = 1:num_iter
        px = p(x)
        x += α*px
        
        cost[iter] = J(x)
        X[:,iter] = x
        P[:,iter] = px
        
        if norm(dQ(x)) <= tau[k]
            k += 1 # Decrease tolerance
            μ *= 2
        end
        if norm(x-x_prev) < 1e-8
            println("Tolerance Condition Hit")
            print_stats()
            return X,P,cost
        end
        x_prev = copy(x)
    end
    iter = num_iter
    print_stats()
    return X,P,cost
end

function plotQ(f,c,μ)
    Q(x) = J(x) + μ/2*c(x)^2
    Q2(x,y) = Q([x;y])
    Xs = linspace(-1.5,1.5,N)
    Ys = linspace(-1.5,1.5,N)
    plt = contourf(Xs,Ys,Q2)
end

function solve_AL(f,ce,ci,x0;num_iter=100,μ=1.,α=0.1)
    function print_stats()
        println("Final Value: $x")
        println("Final Cost: $(L(x))")
        println("Final mu: ", μ)
        println("Final C: $(c(x))")
        println("j: $j/$(length(tau))")
        println("Iterations: $iter")
        println("eps: $(norm(x-x_prev))")
    end
    
    function updateIμ(c, λ)
        for i = Ne+1:N
            if c[i] >= 0 && λ[i] <= 0
                Iμ[i,i] = 0
            end
        end
    end
        
    
    n = length(x0)
    Ne = length(ce(x0))
    Ni = length(ci(x0))
    N = Ne+Ni
    
    tau = logspace(1,-5,30)
    j = 1
    λ = zeros(N)
    if isa(μ,Number)
        Iμ = eye(N)*μ
        println(Iμ)
    else
        Iμ = diagm(μ)
    end
    
    c(x) = [ce(x); ci(x)]
    L(x) = J(x) + c(x)'Iμ*c(x) - λ'c(x)
    dL(x) = ForwardDiff.gradient(L,x)
    hL(x) = ForwardDiff.hessian(L,x)
    p(x) = -inv(hL(x))*dL(x)
    
    updateIμ(c(x0),λ)
    println(Iμ)
    
    x_prev = ones(n)*Inf
    X = zeros(n,num_iter)
    cost = zeros(num_iter)
    P = zeros(n,num_iter)
    x = copy(x0)
    px = zeros(n)
    
    iter_limit = 1000
        
    for iter = 1:num_iter
        for k = 1:iter_limit
            px = p(x)
            x += α*px
            if norm(dL(x)) < tau[j]
                break
            end
        end
        
        j += 1
        λ += -μ*c(x)
        μ *= 2
        
        if norm(x-x_prev) < 1e-8 || abs(c(x)) < 1e-6
            println("Tolerance Condition Hit")
            print_stats()
            return X,P,cost
        end
        
        cost[iter] = J(x)
        X[:,iter] = x
        P[:,iter] = px
        
        x_prev = copy(x)
    end
    iter = num_iter
    print_stats()
    return X,P,cost
end
    


solve_AL (generic function with 2 methods)

In [78]:
# J(x) = sum(x)
# ce(x) = x[1]^2 + x[2]^2 -2
J(x) = x[1]^2 + x[2]^2
ce(x) = x[1] + x[2] - 1
ci(x) = x[2]
n = 2
x0 = [1,-.9]

# X, P, cost = solve_penalty(J,c,num_iter=500);
println()
X, P, cost = solve_AL(J,ce,ci,x0,num_iter=500);
# plotQ(J,c,1)


[1.0 0.0; 0.0 1.0]
[-0.9, -0.9]
[0.0, 0.0]
[1.0 0.0; 0.0 1.0]


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] abs(::Array{Float64,1}) at ./deprecated.jl:57
 [3] #solve_AL#455(::Int64, ::Float64, ::Float64, ::Function, ::Function, ::#ce, ::#ci, ::Array{Float64,1}) at ./In[77]:125
 [4] (::#kw##solve_AL)(::Array{Any,1}, ::#solve_AL, ::Function, ::Function, ::Function, ::Array{Float64,1}) at ./<missing>:0
 [5] include_string(::String, ::String) at ./loading.jl:522
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/bjack205/.julia/v0.6/IJulia/src/execute_request.jl:158
 [7] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/bjack205/.julia/v0.6/Compat/src/Compat.jl:378
 [8] eventloop(::ZMQ.Socket) at /home/bjack205/.julia/v0.6/IJulia/src/eventloop.jl:8
 [9] (::IJulia.##14#17)() at ./task.jl:335
while loading In[78], in expression starting on line 11


LoadError: [91mMethodError: no method matching isless(::Array{Float64,1}, ::Float64)[0m
Closest candidates are:
  isless([91m::Float64[39m, ::Float64) at float.jl:439
  isless([91m::Missings.Missing[39m, ::Any) at /home/bjack205/.julia/v0.6/Missings/src/Missings.jl:74
  isless([91m::AbstractFloat[39m, ::AbstractFloat) at operators.jl:96
  ...[39m

In [95]:
function plot_step(x,px)
    scatter!([x[1]],[x[2]],color="red",legend=:none)
    plot!([x[1], x[1]+px[1]*α],[x[2], x[2]+px[2]*α],width=2,color="black",legend=:none)
end

N = 100
Xs = linspace(-1.5,1.5,N)
Ys = linspace(-1.5,1.5,N)
Q2(x,y) = Q([x;y])
contourf(Xs,Ys,Q2)

@gif for i = 1:size(X,2)
    scatter!([X[1,i]],[X[2,i]],color="red",legend=:none)
end

LoadError: [91mInterruptException:[39m

In [57]:
μ = [1]
isa(μ,Number)

false

makePSD_current (generic function with 1 method)

In [222]:
A = diagm([1, 3, -2])
makePSD_current(A)

[2.0 0.0 0.0; 0.0 4.0 0.0; 0.0 0.0 -1.0]
[4.0 0.0 0.0; 0.0 6.0 0.0; 0.0 0.0 1.0]
3


3×3 Array{Float64,2}:
 4.0  0.0  0.0
 0.0  6.0  0.0
 0.0  0.0  1.0

4×4 Array{Float64,2}:
 9.95217  9.84117  11.2317    6.2585  
 0.0      8.86774   5.98027   4.7625  
 0.0      0.0       4.62761  -0.871035
 0.0      0.0       0.0       1.02907 

Test Passed

4×4 Array{Float64,2}:
 10.166  9.95752  11.1892    6.46353 
  0.0    8.90409   6.04521   4.76489 
  0.0    0.0       4.76655  -0.936592
  0.0    0.0       0.0       1.27698 

4×4 Array{Float64,2}:
 10.166  9.95752  11.1892    6.46353 
  0.0    8.90409   6.04521   4.76489 
  0.0    0.0       4.76655  -0.936592
  0.0    0.0       0.0       1.27698 

search: else elseif ifelse @elapsed fieldoffset SerializationState nfields

search: else elseif ifelse @elapsed fieldoffset SerializationState nfields

search: else elseif ifelse @elapsed fieldoffset SerializationState nfields

